In [7]:
import pandas as pd
import numpy as np

# 1. 定義距離公式 (Haversine)
def haversine_distance(lon1, lat1, lon2, lat2):
    R = 6371000  # 地球半徑 (公尺)
    # 確保輸入轉為浮點數，避免字串運算錯誤
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# 2. 讀取資料 (加入錯誤處理與型別轉換)
file_path = Path(r'C:\Users\user\Desktop\宇統資訊\宇統資訊_車牌辨識系統\LLM_Report_Service_v1\data\realistic_vehicle_dataset1.csv')

# 使用 coerce 強制轉為數字，無法轉的會變 NaN
df = pd.read_csv(file_path)
df['經度'] = pd.to_numeric(df['經度'], errors='coerce')
df['緯度'] = pd.to_numeric(df['緯度'], errors='coerce')

# 移除座標有問題的資料
df = df.dropna(subset=['經度', '緯度'])

# 3. 提取不重複攝影機
unique_cameras = df[['攝影機', '攝影機名稱', '經度', '緯度']].drop_duplicates(subset=['攝影機']).reset_index(drop=True)

print(f"資料讀取成功，共有 {len(unique_cameras)} 支有效座標攝影機。")

# 4. 驗證前兩支攝影機的距離 (泰圳路案例)
cam1 = unique_cameras.iloc[0]
cam2 = unique_cameras.iloc[1]
dist = haversine_distance(cam1['經度'], cam1['緯度'], cam2['經度'], cam2['緯度'])

print("-" * 30)
print(f"【距離驗證測試】")
print(f"攝影機 A: {cam1['攝影機名稱']}")
print(f"攝影機 B: {cam2['攝影機名稱']}")
print(f"計算距離: {dist:.2f} 公尺")
print(f"判斷標準: < 200 公尺? {'YES (應分同一組)' if dist < 200 else 'NO'}")
print("-" * 30)

# 5. 執行分群模擬 (Greedy Clustering)
radius = 200
unique_cameras['LocationAreaID'] = None
area_id_counter = 0
member_count = 0

for i in range(len(unique_cameras)):
    if unique_cameras.loc[i, 'LocationAreaID'] is not None:
        continue
        
    # 建立新群組
    current_area_id = f"Area-{area_id_counter:03d}"
    unique_cameras.loc[i, 'LocationAreaID'] = current_area_id
    center = unique_cameras.iloc[i]
    area_id_counter += 1
    
    # 找尚未分群的
    unassigned = unique_cameras[unique_cameras['LocationAreaID'].isnull()]
    if len(unassigned) > 0:
        # 計算距離
        dists = haversine_distance(center['經度'], center['緯度'], unassigned['經度'], unassigned['緯度'])
        
        # 找出符合條件的索引
        # 注意：dists 是一個 Series，索引對應 unassigned 的索引
        members = unassigned[dists <= radius]
        
        if not members.empty:
            unique_cameras.loc[members.index, 'LocationAreaID'] = current_area_id
            member_count += len(members)

print(f"【分群結果】")
print(f"分群半徑: {radius} 公尺")
print(f"總區域數: {area_id_counter}")
print(f"成員攝影機數 (非中心點): {member_count}")

if member_count > 0:
    print("✅ 成功！程式已正確識別出同一區域內的多支攝影機。")
else:
    print("❌ 失敗，仍然沒有找到成員。")

資料讀取成功，共有 572 支有效座標攝影機。
------------------------------
【距離驗證測試】
攝影機 A: (租11401)泰圳路、泰圳路200巷口-5.往泰圳路222巷(車)
攝影機 B: (租11401)泰圳路、泰圳路200巷口-4.往泰圳路200巷(車)
計算距離: 29.24 公尺
判斷標準: < 200 公尺? YES (應分同一組)
------------------------------
【分群結果】
分群半徑: 200 公尺
總區域數: 169
成員攝影機數 (非中心點): 403
✅ 成功！程式已正確識別出同一區域內的多支攝影機。


In [14]:
import pandas as pd
import numpy as np
from pathlib import Path

def main():
    # 1. 讀取 CSV
    file_path = Path('area_audit_report.csv')
    
    if not file_path.exists():
        print("請先執行上一步的 audit_area_grouping.py")
        return

    print(f"正在分析 {file_path} ...")
    df = pd.read_csv(file_path)
    
    # 【修正處】：使用 str.contains 來包含 "Member (成員)"
    # 只要 Role 欄位裡面包含 "Member" 字眼都算數
    members_df = df[df['Role'].str.contains('Member', na=False)].copy()
    
    if members_df.empty:
        print("錯誤：沒有成員資料。")
        print("偵測到的 Role 種類有：", df['Role'].unique())
        print("請確認上一理產生的 CSV 內容是否正確。")
        return

    distances = members_df['Distance_From_Center']

    # 2. 計算統計量
    mean_val = distances.mean()
    median_val = distances.median()
    q1_val = distances.quantile(0.25)
    q3_val = distances.quantile(0.75)
    max_val = distances.max()

    print("\n" + "="*50)
    print("【 區域擴散度統計分析 (Distance Analysis) 】")
    print("="*50)
    
    print(f"{'樣本數 (N)':<20}: {len(members_df)}")
    print(f"{'平均距離 (Mean)':<20}: {mean_val:.2f} m")
    print(f"{'中位數 (Median)':<20}: {median_val:.2f} m  <-- 最具參考價值")
    print(f"{'第1四分位 (Q1)':<20}: {q1_val:.2f} m")
    print(f"{'第3四分位 (Q3)':<20}: {q3_val:.2f} m")
    print(f"{'最大距離 (Max)':<20}: {max_val:.2f} m")
    print("-" * 30)

    # 3. 判讀建議
    print("\n【 判讀建議 】")
    if median_val > 100:
        print("⚠️  分群可能太鬆散：超過一半的成員距離中心 100m 以上。")
        print("   -> 建議檢查是否將不同路口誤判為同一區。")
    elif q3_val < 100:
        print("✅ 分群緊密：75% 的成員都在 100m 內。")
        print("   -> 200m 的半徑設定目前看起來是安全的。")
    else:
        print("ℹ️  分群適中：大部分成員集中，但仍有部分邊緣成員。")

    # 4. 列出最遠的前 5 個案例供人工檢查
    print("\n【 距離中心最遠的 5 支攝影機 (建議人工檢查) 】")
    # 顯示 Area ID, 成員名稱, 距離
    cols_to_show = ['LocationAreaID', 'Camera_Name', 'Distance_From_Center']
    # 確保這些欄位存在
    cols_to_show = [c for c in cols_to_show if c in members_df.columns]
    
    print(members_df.nlargest(5, 'Distance_From_Center')[cols_to_show].to_string(index=False))

if __name__ == "__main__":
    main()

正在分析 area_audit_report.csv ...

【 區域擴散度統計分析 (Distance Analysis) 】
樣本數 (N)             : 403
平均距離 (Mean)         : 92.76 m
中位數 (Median)        : 81.94 m  <-- 最具參考價值
第1四分位 (Q1)          : 49.33 m
第3四分位 (Q3)          : 140.24 m
最大距離 (Max)          : 199.99 m
------------------------------

【 判讀建議 】
ℹ️  分群適中：大部分成員集中，但仍有部分邊緣成員。

【 距離中心最遠的 5 支攝影機 (建議人工檢查) 】
LocationAreaID                        Camera_Name  Distance_From_Center
      Area-139  (租11401)延平路與金門二街口-4.往金門二街往金門四街(車)                199.99
      Area-038 (租11402)執信一街與英士一街、皓東街口-3.往龍岡路一段(車)                199.48
      Area-027    (租11402)大福路與大新一街口(一)-5.往大新三街(車)                198.27
      Area-021    (租11402)中興路與中興路667巷口-4.往合圳北路(車)                198.12
      Area-105   (租11402)莊敬路、中正路-7.往中正路內車道往南平路(車)                197.78
